In [2]:
'''
Utility function 

'''

import numpy as np

def to_one_hot_enc(arr):

    one_hot_enc = []

    for element in arr:

        if element:
            one_hot_enc.append([1,0])
        else:
            one_hot_enc.append([0,1])
    
    return np.array(one_hot_enc)


def from_one_hot_enc(arr):

    arr_list = []
    
    for element in arr:

        if element[0] > element[1]:
            arr_list.append(1)
        else:
            arr_list.append(0)
    
    return np.array(arr_list)


def reshape_images(arr):

    img_list = []

    for img in arr:
        img = np.reshape(np.array(img),(50,50))
        img_list.append(img)

    return np.array(img_list)


def num_samples_in_classes(arr):

    samples_in_class = np.bincount(arr.astype(int))

    return samples_in_class[0], samples_in_class[1]

In [3]:
'''
Preparing data 

'''

x_training_set = np.load('drive/MyDrive/IST/ML/training_set/Xtrain_Classification_Part1.npy') # (6470, 2500)
y_training_set = np.load('drive/MyDrive/IST/ML/training_set/Ytrain_Classification_Part1.npy') # (6470,)


'''
Evaluate data

'''

# num_of_men, num_of_women = num_samples_in_classes(y_data)

# total = num_of_men + num_of_women

# print('Share of men:   {:.2f}%'.format(num_of_men/total))
# print('Share of women: {:.2f}%'.format(num_of_women/total))
# print('Did you just assume my gender?')

'''
Show image

'''
# import matplotlib.pyplot as plt

# def imshow(img):
#     img = np.reshape(np.array(img),(50,50))
#     plt.imshow(img,cmap='gray')
#     show()

# print(y_data[10])
# imshow(x_data[10])

In [ ]:
pip install keras-tuner --upgrade

In [5]:
from tensorflow import keras
from kerastuner.tuners import RandomSearch

'''

Preparing data
'''

x_training_set = reshape_images(x_training_set)
x_training_set = x_training_set.reshape(6470,50,50,1)

y_training_set = to_one_hot_enc(y_training_set)


'''
Tuning model

'''



def model_builder(hp):
  
  model = keras.Sequential()

  # First layer

  model.add(keras.layers.Conv2D(hp.Choice("First layer", [16, 32, 64]), 3, activation='relu', input_shape=(50,50,1)))

  # Tuning hidden layer

  for i in range(hp.Int("Conv Layers", min_value=0, max_value=3)):
    model.add(keras.layers.Conv2D(hp.Choice(f"layer_{i}_filters", [16,32,64]), 3, activation='relu'))

  # End part

  model.add(keras.layers.MaxPool2D(3,3))

  model.add(keras.layers.Flatten())

  model.add(keras.layers.Dense(hp.Choice("Dense layer", [64, 128, 256, 512, 1024]), activation='relu'),)

  model.add(keras.layers.Dense(2, activation='softmax'))
  
  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['binary_accuracy'])
  

  return model

tuner = RandomSearch(
    model_builder,
    objective='binary_accuracy',
    max_trials=32,
    overwrite=True,
)

stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(x_training_set, y_training_set, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]


Trial 32 Complete [00h 00m 44s]
binary_accuracy: 0.9839644432067871

Best binary_accuracy So Far: 0.9874420166015625
Total elapsed time: 00h 15m 18s
INFO:tensorflow:Oracle triggered exit


In [6]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs

'''
Preparing data

'''

x_training_set = np.load('drive/MyDrive/IST/ML/training_set/Xtrain_Classification_Part1.npy') # (6470, 2500)
y_training_set = np.load('drive/MyDrive/IST/ML/training_set/Ytrain_Classification_Part1.npy') # (6470,)

x_training_set = reshape_images(x_training_set)
x_training_set = x_training_set.reshape(6470,50,50,1)

y_training_set = to_one_hot_enc(y_training_set)

'''
Build model

'''

model   = tuner.hypermodel.build(best_hps)
history = model.fit(x_training_set, y_training_set, epochs=50, validation_split=0.2)

binary_accuracy = history.history['binary_accuracy']
best_epoch = binary_accuracy.index(max(binary_accuracy)) + 1
print('Best epoch: %d' % (best_epoch,))


Epoch 1/50
162/162 [==============================] - 2s 11ms/step - loss: 1.0243 - binary_accuracy: 0.7593 - val_loss: 0.4491 - val_binary_accuracy: 0.8006
Epoch 2/50
162/162 [==============================] - 1s 9ms/step - loss: 0.3339 - binary_accuracy: 0.8572 - val_loss: 0.4237 - val_binary_accuracy: 0.8261
Epoch 3/50
162/162 [==============================] - 1s 8ms/step - loss: 0.2712 - binary_accuracy: 0.8897 - val_loss: 0.3563 - val_binary_accuracy: 0.8547
Epoch 4/50
162/162 [==============================] - 1s 8ms/step - loss: 0.2110 - binary_accuracy: 0.9165 - val_loss: 0.3776 - val_binary_accuracy: 0.8539
Epoch 5/50
162/162 [==============================] - 1s 8ms/step - loss: 0.1777 - binary_accuracy: 0.9301 - val_loss: 0.4018 - val_binary_accuracy: 0.8586
Epoch 6/50
162/162 [==============================] - 1s 8ms/step - loss: 0.1437 - binary_accuracy: 0.9496 - val_loss: 0.3948 - val_binary_accuracy: 0.8640
Epoch 7/50
162/162 [==============================] - 1s 8ms/st

In [9]:
# Train hypermodel with the optimal number of epoch

'''
Preparing data

'''

x_training_set = np.load('drive/MyDrive/IST/ML/training_set/Xtrain_Classification_Part1.npy') # (6470, 2500)
y_training_set = np.load('drive/MyDrive/IST/ML/training_set/Ytrain_Classification_Part1.npy') # (6470,)

x_training_set = reshape_images(x_training_set)
x_training_set = x_training_set.reshape(6470,50,50,1)

y_training_set = to_one_hot_enc(y_training_set)


hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(x_training_set, y_training_set, epochs=best_epoch, validation_split=0.2)


Epoch 1/33
162/162 [==============================] - 2s 11ms/step - loss: 2.0384 - binary_accuracy: 0.7255 - val_loss: 0.4658 - val_binary_accuracy: 0.7913
Epoch 2/33
162/162 [==============================] - 1s 9ms/step - loss: 0.3814 - binary_accuracy: 0.8387 - val_loss: 0.3893 - val_binary_accuracy: 0.8385
Epoch 3/33
162/162 [==============================] - 1s 9ms/step - loss: 0.3123 - binary_accuracy: 0.8752 - val_loss: 0.3865 - val_binary_accuracy: 0.8292
Epoch 4/33
162/162 [==============================] - 1s 9ms/step - loss: 0.2876 - binary_accuracy: 0.8862 - val_loss: 0.3349 - val_binary_accuracy: 0.8640
Epoch 5/33
162/162 [==============================] - 1s 8ms/step - loss: 0.2372 - binary_accuracy: 0.9084 - val_loss: 0.3346 - val_binary_accuracy: 0.8617
Epoch 6/33
162/162 [==============================] - 1s 9ms/step - loss: 0.2080 - binary_accuracy: 0.9169 - val_loss: 0.3566 - val_binary_accuracy: 0.8586
Epoch 7/33
162/162 [==============================] - 1s 8ms/st

In [13]:
# Run test set on model

x_test_set = np.load('drive/MyDrive/IST/ML/test_set/Xtest_Classification_Part1.npy')

x_test_set = reshape_images(x_test_set)
x_test_set = x_test_set.reshape(1164,50,50,1)

pred = hypermodel.predict(x_test_set)

pred = from_one_hot_enc(pred)

np.save("test_set_predictions.npy", pred)

[1 0 0 ... 0 0 0]
